In [1]:
import xml.etree.ElementTree
from svgpathtools import parse_path
from collections import namedtuple
import json
import os
from pathlib import Path
import subprocess
Point = namedtuple('Point', 'x,y')
mult = 8
inkscape = 'C:\Program Files\Inkscape\inkscape'

In [3]:
polygons = {}
for f in Path('svg').glob('*.svg'):
    s = ''
    with f.open('r') as fp:
        s = fp.read()
    with f.open('w') as fp:
        fp.write(s.replace('fill:none;fill-rule:evenodd;stroke:#000000;stroke-width:1px;stroke-linecap:butt;stroke-linejoin:miter;stroke-opacity:1', 'fill:none;fill-rule:evenodd;stroke:none;stroke-width:1px;stroke-linecap:butt;stroke-linejoin:miter;stroke-opacity:1'))
    e = xml.etree.ElementTree.parse(str(f)).getroot()
    size = [float(x) for x in e.get('viewBox').split(' ')][2:4]
    newsize = [x*mult for x in size]
    args = [inkscape, '--without-gui', '--export-png', str((f.parent / "maps" / f.stem).with_suffix('.png')),
            '--export-area-page', '-w', str(int(newsize[0])), '-h', str(int(newsize[1])),
            str(f.with_suffix('.svg'))]
    print(args)
    subprocess.call(args)
    roomlayers = [x for x in e.findall(".//{http://www.w3.org/2000/svg}g") if x.get('id') == 'rooms']
    for roomlayer in roomlayers:
        pathdefs = roomlayer.findall('.//{http://www.w3.org/2000/svg}path')
        for pathdef in pathdefs:
            room = pathdef.get('id')
            path = parse_path(pathdef.get('d'))
            print(room, path)
            polygons[room] = [Point(l.start.real * mult, l.start.imag * mult) for l in path]
with Path('svg/rooms.json').open('w') as f:
    json.dump({k: {"points": v, "name": k} for k, v in polygons.items()}, f)

['C:\\Program Files\\Inkscape\\inkscape', '--without-gui', '--export-png', 'svg\\maps\\D2100.png', '--export-area-page', '-w', '6735', '-h', '4762', 'svg\\D2100.svg']
D2116 Path(Line(start=(423.7069+53.539655j), end=(403.44828+104.18621j)),
     Line(start=(403.44828+104.18621j), end=(458.40517+115.17759000000001j)),
     Line(start=(458.40517+115.17759000000001j), end=(478.23276+62.806897000000006j)),
     Line(start=(478.23276+62.806897000000006j), end=(423.7069+53.539655j)))
path7028 Path(Line(start=(369.18103+44.272414j), end=(354.52586+77.893103j)),
     Line(start=(354.52586+77.893103j), end=(357.11207+78.539655j)),
     Line(start=(357.11207+78.539655j), end=(358.62069+84.789655j)),
     Line(start=(358.62069+84.789655j), end=(351.93966+83.92758599999999j)),
     Line(start=(351.93966+83.92758599999999j), end=(342.24138+105.47931j)),
     Line(start=(342.24138+105.47931j), end=(395.25863+115.39311j)),
     Line(start=(395.25863+115.39311j), end=(420.47414+53.10862099999999j)),
 

In [6]:
polygons['D2265']

[Point(x=2398.50624, y=1370.10168),
 Point(x=2209.0016, y=1797.19416),
 Point(x=2563.9692, y=1880.63272),
 Point(x=2747.81696, y=1446.4692)]

In [43]:
json.dumps(polygons)

'{"path6117": [[7802.892, 2972.8559999999998], [8781.900000000001, 3186.5519999999997], [8639.460000000001, 3733.8599999999997], [7651.416000000001, 3503.58]], "path6121": [[8586.0, 1100.7072], [8516.508, 1363.212], [8793.312, 1412.748], [8861.627999999999, 1149.882]], "D2160_1": [[4340.328, 3576.408], [4886.844, 3720.612], [5016.024, 3365.7839999999997], [5479.103999999999, 3484.2239999999997], [5357.9039999999995, 3846.8759999999997], [7208.939999999999, 4335.456], [6789.1799999999985, 5758.2480000000005], [3825.683999999999, 4877.892000000001]], "path6115": [[7651.416000000001, 3503.58], [8639.460000000001, 3733.8599999999997], [8209.452000000001, 5365.632], [7588.800000000001, 5200.3679999999995], [7744.476000000001, 4611.168], [7366.716, 4501.331999999999]], "path6119": [[8861.628, 1149.882], [9276.900000000001, 1219.32], [9120.240000000002, 1843.56], [8695.72536, 1770.981]], "path6127": [[6027.396000000001, 1476.2640000000001], [5972.268, 1476.0120000000002], [5986.668, 1407.864]

In [14]:
list(Path('svg').glob('*.svg'))[0]

ValueError: Invalid name 'test/t'